In [3]:
import pandas as pd

import common

In [4]:
df = pd.read_csv(common.FINAL_PROFILE_DATABASE)

In [5]:
if df.nvcc_version.unique().shape[0] == 1:
    df = df.drop(columns="nvcc_version")

In [6]:
grouped = df.set_index(["board", "family", "config", "hardening"]).drop(columns=["app"])

In [7]:
hardened = grouped.loc[pd.IndexSlice[:, :, :, "replace-id"]].droplevel("hardening")
unhardened = grouped.loc[pd.IndexSlice[:, :, :, "no-replace-id"]].droplevel("hardening")

## Calculate the diff

In [8]:
diff = pd.concat(
    {
        "cf": (hardened["cf_executed"] - unhardened["cf_executed"]),
        "inst": (hardened["inst_executed"] - unhardened["inst_executed"]),
        "cycles": (hardened["elapsed_cycles_pm"] - unhardened["elapsed_cycles_pm"]),
        "time%": (hardened["execution_time"] - unhardened["execution_time"]) / (
                (hardened["execution_time"] + unhardened["execution_time"]) / 2),
        "inst%": (hardened["inst_executed"] - unhardened["inst_executed"]) / (
                (hardened["inst_executed"] + unhardened["inst_executed"]) / 2)
    },
    axis=1, verify_integrity=True
)

In [9]:
family_diff = diff.groupby(["board", "family"]).agg({"time%": [min, max], "inst%": [min, max]})

In [10]:
family_diff_avg = diff.groupby(["board", "family"]).mean()

In [18]:
family_diff_avg["time%"].mean() * 100

5.613045599086632

In [12]:
family_diff = family_diff.unstack(level="board")


In [13]:
family_diff.columns = family_diff.columns.set_names(["metric", "minmax", "board"])

In [14]:
family_diff = family_diff.stack("minmax").reindex(["min", "max"], level="minmax").unstack(level="minmax").reindex(
    ["pascal", "ampere"], level="board", axis="columns"
).reindex(
    ["time%", "inst%"], level="metric", axis="columns"          
).reindex(
    ["Original", "EVA", "SwinV2", "MaxViT"]
)

In [15]:
fam_d = (family_diff * 100.).round(2)

In [16]:
with pd.ExcelWriter("/home/fernando/git_research/date_2024_vits/data/metrics_tmp.xlsx") as writer:
    grouped.to_excel(writer, sheet_name="raw")
    fam_d.to_excel(writer, sheet_name="fam_avg_diff")